In [1]:
# 데이터 가져오기
import pandas as pd
ratings = pd.read_csv('./Data/ml-latest-small/ratings.csv')
movies = pd.read_csv('./Data/ml-latest-small/movies.csv')

In [2]:
# 두 데이터에서 movieID가 공통 컬럼 -> 데이터 프레임 통합하기 : merge
ratings = pd.merge(ratings, movies, left_on = 'movieId', right_on = 'movieId').drop(['timestamp', 'genres'], axis = 1)

In [3]:
# userId => Index, columns => movieId로 하는 pivot_table 만들기
rp = ratings.pivot_table(index = ['userId'], columns = ['movieId'], values = ['rating'])

In [4]:
# 결측치 -> 0으로 채우기
rp.fillna(0, inplace = True)

In [5]:
# 데이터 프레임 -> array로 변환
import numpy as np
rp_array = rp.to_numpy()

In [24]:
# 계산 시간을 줄이기 위해서 데이터 사이즈 축소
rp.loc['sum', :] = rp.sum(axis = 0)
rp_reduced = rp.transpose().sort_values(by = 'sum', ascending = False).iloc[0:100,:].transpose()

In [26]:
# 데이터 프레임 확인
rp_reduced.head()

rating                                               ...            \
movieId   318  356  296  2571 593  260  110  2959 527  480   ... 733  5445   
userId                                                       ...             
1          0.0  4.0  3.0  5.0  4.0  5.0  4.0  5.0  5.0  4.0  ...  4.0  0.0   
2          3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5  0.0  ...  0.0  0.0   
4          0.0  0.0  1.0  1.0  5.0  5.0  0.0  2.0  0.0  0.0  ...  0.0  0.0   
5          3.0  0.0  5.0  0.0  0.0  0.0  4.0  0.0  5.0  0.0  ...  0.0  0.0   

                                                 
movieId 2628 1527 4878 5349 5418 1968 111  7438  
userId                                           
1        4.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4        1.0  0.0  0.0  0.0  0.0  4.0  0.0  0.0  
5        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 100 columns]

In [28]:
# 데이터 프레임 shape 저장
m,n = rp_reduced.shape

In [30]:
# rp_reduced -> rp_reduced_array
rp_reduced_array = rp_reduced.to_numpy()

In [33]:
# 코사인 유사도 계산하기
from scipy.spatial.distance import cosine
import time
start_time = time.time()
array_movies = np.zeros((n,n))

for i in range(n):
    for j in range(n):
        if i != j:
            array_movies[i,j] = (1-cosine(rp_reduced_array[:,i], rp_reduced_array[:,j]))
            
        else:
            array_movies[i,j] = 0
print(time.time() - start_time)

0.42386603355407715


In [38]:
# 데이터 프레임 만들기
pd_movies = pd.DataFrame(
    array_movies,
    index = rp_reduced.columns,
    columns = rp_reduced.columns
)

In [68]:
# index 수정
index = list()
for i in pd_movies.index:
    index.append(i[1])
pd_movies.index = index

In [69]:
# 데이터 확인
pd_movies

rating                                                              \
movieId      318       356       296       2571      593       260       110    
318      0.000000  0.999981  0.999980  0.999970  0.999975  0.999963  0.999968   
356      0.999981  0.000000  0.999979  0.999972  0.999974  0.999965  0.999972   
296      0.999980  0.999979  0.000000  0.999971  0.999979  0.999965  0.999970   
2571     0.999970  0.999972  0.999971  0.000000  0.999966  0.999973  0.999960   
593      0.999975  0.999974  0.999979  0.999966  0.000000  0.999965  0.999964   
...           ...       ...       ...       ...       ...       ...       ...   
5349     0.999922  0.999925  0.999922  0.999937  0.999919  0.999932  0.999910   
5418     0.999920  0.999921  0.999917  0.999931  0.999907  0.999921  0.999908   
1968     0.999915  0.999920  0.999919  0.999919  0.999916  0.999924  0.999899   
111      0.999913  0.999905  0.999925  0.999905  0.999921  0.999906  0.999902   
7438     0.999918  0.999918  0.999920  0.999926  0.999913  0.999916  0.999904   

                                       ...                                \
movieId      2959      527       480   ...      733       5445      2628   
318      0.999967  0.999969  0.999963  ...  0.999913  0.999924  0.999926   
356      0.999964  0.999967  0.999975  ...  0.999919  0.999925  0.999934   
296      0.999968  0.999963  0.999968  ...  0.999914  0.999923  0.999928   
2571     0.999975  0.999957  0.999957  ...  0.999912  0.999938  0.999945   
593      0.999960  0.999964  0.999965  ...  0.999915  0.999916  0.999925   
...           ...       ...       ...  ...       ...       ...       ...   
5349     0.999928  0.999906  0.999920  ...  0.999882  0.999939  0.999925   
5418     0.999924  0.999897  0.999909  ...  0.999879  0.999929  0.999897   
1968     0.999918  0.999900  0.999915  ...  0.999874  0.999891  0.999901   
111      0.999911  0.999907  0.999896  ...  0.999842  0.999869  0.999863   
7438     0.999938  0.999899  0.999903  ...  0.999862  0.999913  0.999893   

                                                                               
movieId      1527      4878      5349      5418      1968      111       7438  
318      0.999916  0.999913  0.999922  0.999920  0.999915  0.999913  0.999918  
356      0.999923  0.999912  0.999925  0.999921  0.999920  0.999905  0.999918  
296      0.999919  0.999918  0.999922  0.999917  0.999919  0.999925  0.999920  
2571     0.999936  0.999921  0.999937  0.999931  0.999919  0.999905  0.999926  
593      0.999913  0.999909  0.999919  0.999907  0.999916  0.999921  0.999913  
...           ...       ...       ...       ...       ...       ...       ...  
5349     0.999924  0.999900  0.000000  0.999928  0.999889  0.999867  0.999920  
5418     0.999910  0.999893  0.999928  0.000000  0.999880  0.999854  0.999917  
1968     0.999908  0.999884  0.999889  0.999880  0.000000  0.999868  0.999885  
111      0.999858  0.999875  0.999867  0.999854  0.999868  0.000000  0.999875  
7438     0.999903  0.999914  0.999920  0.999917  0.999885  0.999875  0.000000  

[100 rows x 100 columns]

In [71]:
# columns 수정
columns = list()
for i in pd_movies.columns:
    columns.append(i[1])
pd_movies.columns = columns    

In [72]:
# 데이터 확인하기
pd_movies

,318,356,296,2571,593,260,110,2959,527,480,...,733,5445,2628,1527,4878,5349,5418,1968,111,7438
318,0.000000,0.999981,0.999980,0.999970,0.999975,0.999963,0.999968,0.999967,0.999969,0.999963,...,0.999913,0.999924,0.999926,0.999916,0.999913,0.999922,0.999920,0.999915,0.999913,0.999918
356,0.999981,0.000000,0.999979,0.999972,0.999974,0.999965,0.999972,0.999964,0.999967,0.999975,...,0.999919,0.999925,0.999934,0.999923,0.999912,0.999925,0.999921,0.999920,0.999905,0.999918
296,0.999980,0.999979,0.000000,0.999971,0.999979,0.999965,0.999970,0.999968,0.999963,0.999968,...,0.999914,0.999923,0.999928,0.999919,0.999918,0.999922,0.999917,0.999919,0.999925,0.999920
2571,0.999970,0.999972,0.999971,0.000000,0.999966,0.999973,0.999960,0.999975,0.999957,0.999957,...,0.999912,0.999938,0.999945,0.999936,0.999921,0.999937,0.999931,0.999919,0.999905,0.999926
593,0.999975,0.999974,0.999979,0.999966,0.000000,0.999965,0.999964,0.999960,0.999964,0.999965,...,0.999915,0.999916,0.999925,0.999913,0.999909,0.999919,0.999907,0.999916,0.999921,0.999913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5349,0.999922,0.999925,0.999922,0.999937,0.999919,0.999932,0.999910,0.999928,0.999906,0.999920,...,0.999882,0.999939,0.999925,0.999924,0.999900,0.000000,0.999928,0.999889,0.999867,0.999920
5418,0.999920,0.999921,0.999917,0.999931,0.999907,0.999921,0.999908,0.999924,0.999897,0.999909,...,0.999879,0.999929,0.999897,0.999910,0.999893,0.999928,0.000000,0.999880,0.999854,0.999917
1968,0.999915,0.999920,0.999919,0.999919,0.999916,0.999924,0.999899,0.999918,0.999900,0.999915,...,0.999874,0.999891,0.999901,0.999908,0.999884,0.999889,0.999880,0.000000,0.999868,0.999885
111,0.999913,0.999905,0.999925,0.999905,0.999921,0.999906,0.999902,0.999911,0.999907,0.999896,...,0.999842,0.999869,0.999863,0.999858,0.999875,0.999867,0.999854,0.999868,0.000000,0.999875


In [83]:
# 유사한 영화 검색
def topn_simovies(mid = 318, n = 15):
    mid_ratings = pd_movies.loc[mid, :].sort_values(ascending = False)
    topn_movies = pd.DataFrame(mid_ratings.iloc[:n])
    topn_movies['index1'] = topn_movies.index
    topn_movies['index1'] = topn_movies['index1'].astype('int64')
    topn_movies = pd.merge(topn_movies, movies[['movieId', 'title']], left_on = 'index1', right_on = 'movieId')
    del topn_movies['index1']
    return topn_movies

In [84]:
topn_simovies(mid = 111, n=15)

,111,movieId,title
0,0.999927,608,Fargo (1996)
1,0.999925,296,Pulp Fiction (1994)
2,0.999921,858,"Godfather, The (1972)"
3,0.999921,593,"Silence of the Lambs, The (1991)"
4,0.999916,1208,Apocalypse Now (1979)
5,0.999915,1089,Reservoir Dogs (1992)
6,0.999913,318,"Shawshank Redemption, The (1994)"
7,0.999911,2959,Fight Club (1999)
8,0.999910,2858,American Beauty (1999)
9,0.999910,50,"Usual Suspects, The (1995)"
